In [ ]:
from scipy.io import loadmat
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import random
from sklearn.model_selection import train_test_split
py.init_notebook_mode()

#importar funcion "plot" definida en "utils.py"
from utils import plot


# tensorflow
import tensorflow as tf
from tensorflow import layers

In [ ]:
#cargar archivo
mat = loadmat("data/mnist.mat")
mat

In [ ]:
#assignar data
x = mat['X']
y = mat['y'] % 10


x.shape, y.shape

In [ ]:
#visualizar archivos algunas muestras
for _ in range(1):
    n = random.randint(0, len(x) - 1)
    title = "Digit: {}".format(y[n][0])
    plot(x[n], title, width=400, height=400)

In [ ]:
class Model(object):
    
    def __init__(self, learning_rate=0.01):
        
        self.graph = tf.Graph()
        
        with self.graph.as_default():
            self.x = tf.placeholder(tf.float32, [None, 400])
            self.y = tf.placeholder(tf.int64, [None, 1])

            net = layers.dense(self.x, 256, activation=tf.nn.relu)
            net = layers.dense(net, 128, activation=tf.nn.relu)
            net = layers.dense(net, 64, activation=tf.nn.relu)

            logits = layers.dense(net, 10, activation=None)

            self.h = tf.nn.softmax(logits)
            self.prediction = tf.argmax(self.h, axis=1)

            labels = tf.one_hot(self.y, 10)
            self.loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
            self.loss = tf.reduce_mean(self.loss)
            
            self.update = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
            
            y1 = tf.expand_dims(tf.argmax(self.h, axis=1), 1)
            self.accuracy = tf.equal(y1, self.y)
            self.accuracy = tf.cast(self.accuracy, tf.float32)
            self.accuracy = tf.reduce_mean(self.accuracy)
            
            self.sess = tf.Session(graph=self.graph)
            self.sess.run(tf.global_variables_initializer())
            
    
            
    def fit(self, x, y, epochs=2000, print_interval=100, batch_size=64):
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
        
        for step in xrange(epochs):
            
            idx = random.sample(range(len(x_train)), batch_size)
            x_batch, y_batch = x_train[idx], y_train[idx]
            
            _, loss = self.sess.run([self.update, self.loss], feed_dict={self.x: x_batch, self.y: y_batch})
            
            if step % print_interval == 0:
                accuracy = self.sess.run(self.accuracy, feed_dict={self.x: x_test, self.y: y_test})
                print "loss: {0}, accuracy: {1}".format(loss, accuracy)
                
                
    def predict(self, x):
        return self.sess.run(self.prediction, feed_dict={self.x: x})
        
        
        
model = Model()

In [ ]:
model.fit(x, y)

In [ ]:
n = random.randint(0, len(x) - 1)

x_sample, y_sample = x[n:n+1, :], y[n][0]

prediction = model.predict(x_sample)[0]
title = "Digit: {0}, prediction: {1}".format(y_sample,prediction)

plot(x_sample, title, width=400, height=400)

In [ ]:
%%bash
#cat /run_jupyter.sh